In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import numba
from scipy import integrate, sparse, linalg, interpolate

# Field of the point charge

In [ ]:
gamma_0 = 20
v0 = np.sqrt(1 - gamma_0 ** -2)

In [ ]:
A_z = lambda xi, r: gamma_0 * v0 / 4 / np.pi / np.sqrt(gamma_0 ** 2 * xi ** 2 + r ** 2)

In [ ]:
xi = np.linspace(-2, 2, 300)
r = np.linspace(0, 2, 300)
xx, yy = np.meshgrid(xi, r)

In [ ]:
plt.pcolormesh(xx, yy, A_z(xx, yy), vmin=-0.1, vmax=0.1, cmap='RdBu_r')

In [ ]:
E_z = lambda xi, r: gamma_0 * xi / 4 / np.pi / (gamma_0 ** 2 * xi ** 2 + r ** 2) ** 1.5

In [ ]:
plt.pcolormesh(xx, yy, E_z(xx, yy), vmin=-0.1, vmax=0.1, cmap='RdBu_r')

In [ ]:
E_r = lambda xi, r: gamma_0 * r / 4 / np.pi / (gamma_0 ** 2 * xi ** 2 + r ** 2) ** 1.5

In [ ]:
plt.pcolormesh(xx, yy, E_r(xx, yy), vmin=-0.1, vmax=0.1, cmap='RdBu_r')

In [ ]:
B_phi = lambda xi, r: - v0 * E_r (xi, r)

In [ ]:
plt.pcolormesh(xx, yy, B_phi(xx, yy), vmin=-0.1, vmax=0.1, cmap='RdBu_r')

# Bunch

In [ ]:
gamma0 = 3
v0 = np.sqrt(1 - gamma_0 ** -2)

@numba.jit
def distance_sq(xi, xi1, r, r1, phi):
    return gamma0 ** 2 * (xi - xi1) ** 2 + r ** 2 + r1 ** 2 - 2 * r * r1 * math.cos(phi)

def integrate_over_bunch(func, xi=0., r=0., halflength=1., radius=1.):
    return integrate.tplquad(numba.jit(lambda phi, r1, xi1: func(xi, xi1, r, r1, phi)),
                             -halflength, halflength,
                             lambda x: 0., lambda x: radius,
                             lambda x,y: 0., lambda x,y: 2 * np.pi)[0]

def potential(xi, r):
    integrand = numba.jit(lambda xi, xi1, r, r1, phi: gamma0 / 4 / math.pi * r1 / math.sqrt(distance_sq(xi, xi1, r, r1, phi)))
    return integrate_over_bunch(integrand, xi, r)

In [ ]:
potential(0., 1.)

In [ ]:
r = np.linspace(0, 1,)

# Poisson solver

In [ ]:
import sys
sys.path.append('..')
import poisson

In [ ]:
gamma0 = 10
solver = poisson.PoissonCylindric(zmin = -7, zmax = 7, rmax = 15, dr = 0.025, dz = 0.0125, gamma0=gamma0)

In [ ]:
#func = np.vectorize(lambda x, r: (1-x**2) * (1-r**2) if np.abs(x) < 1 and r < 1 else 0.0)
@np.vectorize
def func(x, r):
    if np.abs(x) > 1:
        return 0.0
    else:
        if r < 1:
            return (1 - x ** 2)
        elif r < np.sqrt(2):
            return - (1 - x ** 2)
        else:
            return 0
    
x, r, u = solver.solve(func)

In [ ]:
plt.pcolormesh(x, r, func(x,r), cmap='RdBu_r')
plt.colorbar()

In [ ]:
def plot_dist(f):
    plt.pcolormesh(x, r, f, vmin=-np.abs(f).max(), vmax=np.abs(f).max(), cmap='RdBu_r')
    plt.colorbar()

plot_dist(u)

In [ ]:
func = np.vectorize(lambda z: -gamma0 * 0.5 * integrate.quad(lambda z1: math.sqrt(1 + gamma0 **2 * (z - z1) ** 2) - gamma0 * abs(z - z1), -1, 1)[0])
plt.plot(solver.z1d, u[0])
plt.plot(solver.z1d, func(solver.z1d))

In [ ]:
def dist_func(f):
    return np.vectorize(interpolate.interp2d(solver.z1d, solver.r1d, f))

In [ ]:
Az = - solver.v0 * u
plot_dist(Az)

In [ ]:
plt.plot(solver.r1d, dist_func(Az)(0, solver.r1d))
plt.plot(solver.r1d, dist_func(Az)(0.5, solver.r1d))
plt.plot(solver.r1d, dist_func(Az)(1, solver.r1d))
plt.plot(solver.r1d, dist_func(Az)(1.5, solver.r1d))

In [ ]:
dur, duz = np.gradient(u, solver.dr, solver.dz)

In [ ]:
Ez = duz / gamma0 ** 2
plot(Ez)

In [ ]:
Er = dur
plot(Er)

In [ ]:
#field_func = np.vectorize(lambda r: 0.5 * r if r < 1 else 0.5 / r)
#field_func = np.vectorize(lambda r: 0.25 * r ** 2 * (2 - r ** 2) if r < 1 else 0.25 / r)
@np.vectorize
def field_func(r):
    if r < 1:
        return 0.5 * r
    elif r < np.sqrt(2):
        return 1 / np.sqrt(r) - r / 2
    else:
        return 0
plt.plot(solver.r1d, dist_func(Er)(0, solver.r1d))
plt.plot(solver.r1d, field_func(solver.r1d), '--')

In [ ]:
plt.contour(x, r, np.sqrt(Ez ** 2 + Er ** 2), levels=[0.01, 0.05, 0.1])

In [ ]:
Bphi = - solver.v0 * dur
plot(Bphi)

In [ ]:
plt.plot(solver.r1d, dist_func(Bphi)(0.0, solver.r1d))
plt.plot(solver.r1d, -field_func(solver.r1d), '--')

In [ ]:
@np.vectorize
def pot_func(r):
    if r > np.sqrt(2):
        return 0.
    elif r > 1:
        return -(np.log(np.sqrt(2) / r) - 0.5 + 0.25 * r ** 2)
    else:
        return -(np.log(np.sqrt(2)) - 0.25 * r ** 2)

plt.plot(solver.r1d, dist_func(u)(0, solver.r1d))
plt.plot(solver.r1d, pot_func(solver.r1d))

In [ ]:
pot_func(1.1)

# Exact solution at (0,0)

In [ ]:
def density(x, r):
    if abs(x) > 1:
        return 0.0
    else:
        if r < 1.0:
            return (1.0 - x ** 2)
        elif r < math.sqrt(2):
            return - (1.0 - x ** 2)
        else:
            return 0.0

In [ ]:
%timeit np.vectorize(density)(x, r)

In [ ]:
plt.pcolormesh(x, r, np.vectorize(density)(x, r))

In [ ]:
d = numba.jit(density, nopython=True)
%timeit d(0.0, 0.0)
%timeit density(0.0, 0.0)

In [ ]:
def answer(gamma0, x0):
    return -0.5 * gamma0 * integrate.dblquad(
        lambda r, x: density(x, r) * r / math.sqrt(gamma0 ** 2 * (x - x0) ** 2 + r**2), 
        -1, 1, lambda x:0, lambda x:math.sqrt(2))[0]

In [ ]:
answer(10,0)

In [ ]:
np.log(np.sqrt(2))

In [ ]:
x0 = np.linspace(-1.5, 1.5, 100)
plt.plot(x0, [answer(10, x) for x in x0])
plt.plot(x0, -np.vectorize(lambda x: (1 - x0 **2) if x < 1 else 0.0)(x0) * np.log(np.sqrt(2)))

In [ ]:
%timeit answer(10, 0)